# **Simulation in closed-loop**

In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

import package_LAB
from importlib import reload
package_LAB = reload(package_LAB)

from package_LAB import LL_RT, PID_RT, IMCTuning
from package_DBR import SelectPath_RT, Delay_RT, FO_RT


In [220]:
#plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interactive, VBox, IntRangeSlider

## Simulation parameters

In [221]:
TSim = 1000
Ts = 0.5    
N = int(TSim/Ts) + 1 

## Dynamics and controller parameters

In [222]:
#DV 
Kp_ODV_SOPDT = 0.2951290424136788
T1_ODV_SOPDT = 182.2549613489765
T2_ODV_SOPDT = 13.184430234847984
theta_ODV_SOPDT = 28.999891911961512

#MV
Kp_OMV_SOPDT = 0.30788564834253684
T1_OMV_SOPDT = 183.81942938046797
T2_OMV_SOPDT = 3.2920224028341535e-12
theta_OMV_SOPDT = 20.015407110302775

#Operating points 
DV0 = 50 
MV0 = 50
PV0 = 49.3

# Set maximum and minimum MV values
MVmin = 0
MVmax = 100

## Scenarios

In [223]:
# Scenarios :
# Path for SP, MAN, MV_MAN, FF

scenario = 6 #1 to 6

if scenario == 1:
    # scenario 1: Change in set point
    ManPath = {0: True, 180: False}
    MVmanPath = {0: 50}
    SPPath = {0: PV0, 250: PV0+10}
    FFPath = {0: False}
    DVPATh = {0:50}
elif scenario == 2:
    # scenario 2: Manual mode no FF, response to perturbation
    ManPath = {0: True}
    MVmanPath = {0: 50}
    SPPath = {0: PV0}
    FFPath = {0: False}
    DVPATh = {0:50, 500: 70}
elif scenario == 3:
    # scenario 3: Manual mode with FF, response to perturbation
    ManPath = {0: True}
    MVmanPath = {0: 50}
    SPPath = {0: PV0}
    FFPath = {0: True}
    DVPATh = {0:50, 400: 70}
elif scenario == 4:
    # scenario 4: Automatic mode no FF, response to perturbation
    ManPath = {0: False}
    MVmanPath = {0: 50}
    SPPath = {0: PV0}
    FFPath = {0: False}
    DVPATh = {0:50, 300: 70}
elif scenario == 5:
    # scenario 5: Automatic mode with FF, response to perturbation
    ManPath = {0: False}
    MVmanPath = {0: 50}
    SPPath = {0: PV0}
    FFPath = {0: True}
    DVPATh = {0:50, 300: 70}
elif scenario == 6:
    # scenario 6: Automatic mode with FF, response to set point change and perturbation
    SPPath = {0:PV0, 400: PV0+10}
    ManPath = {0: False}
    MVmanPath = {0: 0}
    FFPath = {0: True}
    DVPATh = {0:50, 650:30}

## Running simulation

### Initialization of arrays and parameters

In [224]:
t = []
SP = []
PV = []
Man = []
MVMan = []
FF = []
PV_P = []
DV = []
PV_D = []
MVFF = []

MV = []
MVp = []
MVi = []
MVd = []
E = []

#intermediate MV arrays
MV_P_delay = []
MV_P_FO_delay = []
MVFF_delay = []
MVFF_LL = []
MV_D_delay = []
MV_D_FO_delay = []

#global parameters
alphaP = 0.1
Kc = 1
TI = 1
TD = 1
PVInit = 49.3
ManFF = False

for i in range(1, N):    
    t.append(i*Ts)
    SelectPath_RT(SPPath,t,SP)
    SelectPath_RT(ManPath,t,Man)
    SelectPath_RT(MVmanPath,t,MVMan) 
    SelectPath_RT(FFPath,t,FF)
    SelectPath_RT(DVPATh,t,DV)
    
    #Feed Forward
    Delay_RT(DV-DV0*np.ones_like(DV), np.max([theta_ODV_SOPDT-theta_OMV_SOPDT,0]), Ts, MVFF_delay)
    LL_RT(MVFF_delay, -Kp_OMV_SOPDT/Kp_ODV_SOPDT, T1_OMV_SOPDT, T1_ODV_SOPDT, Ts, MVFF_LL)
    LL_RT(MVFF_LL,1,T2_OMV_SOPDT, T2_ODV_SOPDT,Ts,MVFF)
    
    #PID
    PID_RT(SP[0:i], PV[0:i], Man[0:i], MVMan[0:i], MVFF[0:i], Kc, TI, TD, alphaP, Ts, MVmin, MVmax, MV, MVp, MVi, MVd, E, ManFF=ManFF)
    
    #Process
    Delay_RT(MV,theta_OMV_SOPDT,Ts, MV_P_delay, MV0) 
    FO_RT(MV_P_delay, Kp_OMV_SOPDT, T1_OMV_SOPDT, Ts, MV_P_FO_delay, 0)
    FO_RT(MV_P_FO_delay, 1, T2_OMV_SOPDT, Ts, PV_P, 0)
        
    #Disturbance
    Delay_RT(DV-DV0*np.ones_like(DV), theta_ODV_SOPDT, Ts, MV_D_delay, 0)
    FO_RT(MV_D_delay, Kp_ODV_SOPDT, T1_ODV_SOPDT, Ts, MV_D_FO_delay, 0)
    FO_RT(MV_D_FO_delay, 1, T2_ODV_SOPDT, Ts,PV_D, 0)
    
    PV.append(PV_P[-1] + PV_D[-1] + PV0 - Kp_OMV_SOPDT * MV0)   


### Plot code

In [225]:
# Create figure
fig = go.FigureWidget(make_subplots(rows=3, cols=1, specs = [[{}], [{}], [{}]], vertical_spacing = 0.2, row_heights=[0.4, 0.4, 0.2], subplot_titles=("PV, SP and E", "MV and Components", "Manual Mode")))
fig.add_trace(go.Scatter(x=t, y=SP, name="SP"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=PV, name="PV"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=DV, name="DV"), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=E, name="E", line=dict(dash='dash')), row=3, col=1)
fig.add_trace(go.Scatter(x=t, y=MV, name="MV"), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVp, name="MVp", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVi, name="MVi", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=MVd, name="MVd", line=dict(dash='dash')), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=Man, name="Man"), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=MVMan, name="MVMan"), row=1, col=1)

# Update layout
fig['layout'].update(height=800, width=800)
fig['layout']['xaxis1'].update(title='Time (s)')
fig['layout']['yaxis1'].update(title='(°C)')
fig['layout']['xaxis2'].update(title='Time (s)')
fig['layout']['yaxis2'].update(title='MV (%)')
fig['layout']['xaxis3'].update(title='Time (s)')

def UpdatePlot():
    t =[]
    SP = []
    PV = []
    Man = []
    MVMan = []
    FF = []
    PV_P = []
    DV = []
    PV_D = []
    MVFF = []

    MV = []
    MVp = []
    MVi = []
    MVd = []
    E = []

    #intermediate MV arrays
    MV_P_delay = []
    MV_P_FO_delay = []
    MVFF_delay = []
    MVFF_LL = []
    MV_D_delay = []
    MV_D_FO_delay = []
    
    for i in range(1, N):    
        t.append(i*Ts)
        SelectPath_RT(SPPath,t,SP)
        SelectPath_RT(ManPath,t,Man)
        SelectPath_RT(MVmanPath,t,MVMan) 
        SelectPath_RT(FFPath,t,FF)
        SelectPath_RT(DVPATh,t,DV)
        
        #Feed Forward
        Delay_RT(DV-DV0*np.ones_like(DV), np.max([theta_ODV_SOPDT-theta_OMV_SOPDT,0]), Ts, MVFF_delay)
        LL_RT(MVFF_delay, -Kp_OMV_SOPDT/Kp_ODV_SOPDT, T1_OMV_SOPDT, T1_ODV_SOPDT, Ts, MVFF_LL)
        LL_RT(MVFF_LL,1,T2_OMV_SOPDT, T2_ODV_SOPDT,Ts,MVFF)
        
        #PID
        PID_RT(SP[0:i], PV[0:i], Man[0:i], MVMan[0:i], MVFF[0:i], Kc, TI, TD, alphaP, Ts, MVmin, MVmax, MV, MVp, MVi, MVd, E, ManFF=ManFF)
        
        #Process
        Delay_RT(MV,theta_OMV_SOPDT,Ts, MV_P_delay, MV0) 
        FO_RT(MV_P_delay, Kp_OMV_SOPDT, T1_OMV_SOPDT, Ts, MV_P_FO_delay, 0)
        FO_RT(MV_P_FO_delay, 1, T2_OMV_SOPDT, Ts, PV_P, 0)
            
        #Disturbance
        Delay_RT(DV-DV0*np.ones_like(DV), theta_ODV_SOPDT, Ts, MV_D_delay, 0)
        FO_RT(MV_D_delay, Kp_ODV_SOPDT, T1_ODV_SOPDT, Ts, MV_D_FO_delay, 0)
        FO_RT(MV_D_FO_delay, 1, T2_ODV_SOPDT, Ts,PV_D, 0)
        
        PV.append(PV_P[-1] + PV_D[-1] + PV0 - Kp_OMV_SOPDT * MV0)
        
    fig.data[0].y = SP
    fig.data[1].y = PV
    fig.data[2].y = DV
    fig.data[3].y = E
    fig.data[4].y = MV
    fig.data[5].y = MVp
    fig.data[6].y = MVi
    fig.data[7].y = MVd
    fig.data[8].y = Man
    fig.data[9].y = MVMan
    
    
    

def Update_Gamma(gamma):
    global Kc
    global TI
    global TD
    
    Kc, TI, TD = IMCTuning(Kp_OMV_SOPDT, T1_OMV_SOPDT, T2_OMV_SOPDT, theta_OMV_SOPDT, gamma, model='FOPDT')
    UpdatePlot()

def Update_Alpha(alpha):
    global alphaP
    
    alphaP = alpha
    UpdatePlot()
    
def Update_Manual(manual, MVManual, ActiveManual):
    global ManPath
    global MVmanPath
    global ManFF
     
    if ActiveManual:
        ManPath = {0: False, manual[0]: True, manual[1]: False}
        MVmanPath = {0: 0, manual[0]: MVManual, manual[1]: 0}
        ManFF = True; 
    else :
        ManFF = False
        ManPath = {0: False}
        MVmanPath = {0: 0}
    
    t = []
    for i in range(1, N):
        t.append(i*Ts)
        SelectPath_RT(ManPath,t,Man)
        SelectPath_RT(MVmanPath,t,MVMan) 
        
    UpdatePlot()
        

def Update_FF(ActiveFF):
    global FFPath
    
    if not ActiveFF :
        FFPath = {0: False}
    else : 
        FFPath = {0: True}
        
    UpdatePlot()

        
# Create interactive widget
GammaSlider = interactive(Update_Gamma, gamma=(0.2, 0.9, 0.02))
AlphaSlider = interactive(Update_Alpha, alpha=(0, 0.9, 0.01))
ManualSlider = interactive(Update_Manual, manual=IntRangeSlider(min=0, max=100, step=1, value=[1, 999]), MVManual=(0, 100, 1), ActiveManual=False)
FFCheckBox = interactive(Update_FF, ActiveFF=False)
    
# Display figure and widget
vb = VBox([fig, ManualSlider, GammaSlider, AlphaSlider, FFCheckBox])
vb.layout.align_items = 'center'
vb  
        

    'data': [{'name': 'SP',
              'type': 'scatter',
              'uid'…